In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np

from sklearn import metrics

import joblib

2024-02-26 01:05:23.307809: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 01:05:23.311055: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 01:05:23.342736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 01:05:23.342790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 01:05:23.344422: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_test.npy")
x_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_train.npy")
x_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_val.npy")
y_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_test.npy")
y_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_train.npy")
y_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_val.npy")

In [4]:
model = joblib.load("/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib")

2024-02-26 01:06:55.723222: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://2a0416f1d08d4cca9886602842710f82: INVALID_ARGUMENT: ram://2a0416f1d08d4cca9886602842710f82 is a directory.


In [5]:
x_test_trimmed = x_test[20000:24000]
y_test_trimmed = y_test[20000:24000]

In [6]:
np.unique(y_test_trimmed)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [7]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=19,
    input_shape=(58,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [22]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.01, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_277/3492240313.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [10:36<00:00,  6.29it/s]


In [9]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 828us/step
Accuracy: 0.699
AUC: 0.9769664644631458

macro
Precision: 0.4355494350500188
Recall: 0.3827848686120252
F1 Score: 0.3399781437475444

weighted
Precision: 0.7989048305639614
Recall: 0.699
F1 Score: 0.7245349044966327

Mean FNR: 0.6172151313879747
Mean TNR: 0.9782429851469919
Mean FPR: 0.021757014853008246
Mean TPR: 0.3827848686120252


In [10]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.05, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_277/75040551.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [03:00<00:00, 22.20it/s]


In [11]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 696us/step
Accuracy: 0.745
AUC: 0.9699711190490176

macro
Precision: 0.5753778536023106
Recall: 0.38713430073791094
F1 Score: 0.36077922369625326

weighted
Precision: 0.7867097898671599
Recall: 0.745
F1 Score: 0.7489278714056823

Mean FNR: 0.612865699262089
Mean TNR: 0.9807811222605558
Mean FPR: 0.019218877739444244
Mean TPR: 0.38713430073791094


In [12]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.1, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_277/547066443.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [02:13<00:00, 30.02it/s]


In [13]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 734us/step
Accuracy: 0.52425
AUC: 0.9164696861977167

macro
Precision: 0.4200629001906489
Recall: 0.3513270214942508
F1 Score: 0.26594619618766085

weighted
Precision: 0.6878442673694228
Recall: 0.52425
F1 Score: 0.5736573455228817

Mean FNR: 0.6486729785057491
Mean TNR: 0.9663986280353588
Mean FPR: 0.03360137196464117
Mean TPR: 0.3513270214942508


In [14]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.15, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_277/971322502.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:59<00:00, 33.40it/s]


In [15]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 733us/step
Accuracy: 0.524
AUC: 0.896323121566107

macro
Precision: 0.411286220434215
Recall: 0.2863218047372434
F1 Score: 0.2445281095646043

weighted
Precision: 0.6730857064276071
Recall: 0.524
F1 Score: 0.571539333697314

Mean FNR: 0.7136781952627566
Mean TNR: 0.9665186787293975
Mean FPR: 0.033481321270602625
Mean TPR: 0.2863218047372434


In [16]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.2, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_277/1386664024.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:52<00:00, 35.69it/s]


In [17]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 770us/step
Accuracy: 0.548
AUC: 0.8693032838371115

macro
Precision: 0.2421162549254335
Recall: 0.2819109740012041
F1 Score: 0.2014043419666922

weighted
Precision: 0.6338276145429536
Recall: 0.548
F1 Score: 0.5657287886113289

Mean FNR: 0.718089025998796
Mean TNR: 0.9659624619162119
Mean FPR: 0.034037538083788156
Mean TPR: 0.2819109740012041


In [18]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.25, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_277/398718751.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:42<00:00, 39.19it/s]


In [19]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 650us/step
Accuracy: 0.4995
AUC: 0.8497330197943184

macro
Precision: 0.2526717638899624
Recall: 0.18457364858410208
F1 Score: 0.1636539537482599

weighted
Precision: 0.5576842990393567
Recall: 0.49949999999999994
F1 Score: 0.515654790427835

Mean FNR: 0.8154263514158979
Mean TNR: 0.9603261250835569
Mean FPR: 0.039673874916443086
Mean TPR: 0.18457364858410208


In [20]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma = 0.1)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_277/2298617831.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [01:33<00:00, 42.94it/s]


In [21]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 702us/step
Accuracy: 0.3995
AUC: 0.7924743534656578

macro
Precision: 0.31500132887518073
Recall: 0.12223783467646311
F1 Score: 0.12412778727542857

weighted
Precision: 0.4824327416581432
Recall: 0.3995000000000001
F1 Score: 0.41847308695521507

Mean FNR: 0.8777621653235369
Mean TNR: 0.9498722234586282
Mean FPR: 0.0501277765413717
Mean TPR: 0.12223783467646311
